In [4]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

# Load .env
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("❌ GEMINI_API_KEY is missing in .env")

# Configure Gemini
genai.configure(api_key=api_key)

# Create the model (pick a valid model from your list)
model = genai.GenerativeModel(
    "gemini-2.5-flash",   # or "gemini-pro-latest", etc. from your ListModels output
    system_instruction=(
        "You are a police sketch artist. "
        "Ask one focused, concise follow-up question at a time to refine the description "
        "of a person based on the user's answers."
    ),
)

print("✅ Gemini model is ready.")

✅ Gemini model is ready.


In [5]:
import textwrap
import ipywidgets as widgets
from IPython.display import display

# Conversation memory
chat_history = []

# ── Widgets ──────────────────────────────────────────────────────────────
# Text input (no description to avoid layout bugs)
input_box = widgets.Text(
    placeholder="Describe a feature… or type 'history' / 'exit'",
    layout=widgets.Layout(width='70%'),
    style={'description_width': '0px'}  # make sure label doesn't steal space
)

# Separate label "You:" so it's always visible
you_label = widgets.Label("You:")

send_button = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='10%')
)

# Scrollable chat area
output_area = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        height='400px',        # fixed height
        overflow_y='auto',     # vertical scroll
        border='1px solid #ddd',
        padding='6px'
    )
)

def print_history():
    """Print full history into the scrollable area."""
    with output_area:
        print("\n-- Chat History --")
        if not chat_history:
            print("No history yet.")
        else:
            for i, (u, b) in enumerate(chat_history, start=1):
                print(f"{i}. You: {u}\n   Bot: {b}\n")
        print("-----------------------\n")

def on_send_clicked(_):
    text = input_box.value.strip()
    if not text:
        return

    # Clear input
    input_box.value = ""

    lower = text.lower()

    # Exit / quit
    if lower in ("exit", "quit"):
        with output_area:
            print(f"\nYou: {text}")
            print("Bot: Goodbye 👋 Chat ended.")
        input_box.disabled = True
        send_button.disabled = True
        return

    # History
    if lower == "history":
        with output_area:
            print(f"\nYou: {text}")
        print_history()
        return

    # Normal message
    with output_area:
        print(f"\nYou: {text}")
        print("Bot: (thinking…)\n")

    # Build prompt with context
    history_text = "\n".join([f"- {u}" for u, _ in chat_history])
    if history_text:
        full_prompt = (
            f"Previous details about the individual:\n{history_text}\n\n"
            f"New detail: {text}\n\n"
            "As a police sketch artist, respond with ONE short, focused follow-up or "
            "clarification question to refine the description."
        )
    else:
        full_prompt = (
            f"Detail: {text}\n\n"
            "You are a police sketch artist. Ask ONE focused question to refine the description."
        )

    # Call Gemini
    try:
        response = model.generate_content(full_prompt)
        answer = response.text
    except Exception as e:
        answer = f"❌ Error while calling Gemini: {e}"

    # Wrap long responses so they don't overflow horizontally
    wrapped_answer = textwrap.fill(answer, width=100)

    # Save to history
    chat_history.append((text, wrapped_answer))

    # Print answer
    with output_area:
        print(f"Bot: {wrapped_answer}\n")

send_button.on_click(on_send_clicked)

# Put label + input + button on same row
top_row = widgets.HBox([you_label, input_box, send_button])

display(top_row, output_area)

Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…